In [1]:
import os
import os.path
import sys
import json
import random
# sys.path.append(os.getcwd())
sys.path.append('../')
from easyeditor import LoRAHyperParams
from easyeditor import BaseEditor
from easyeditor.models.ike import encode_ike_facts
from sentence_transformers import SentenceTransformer
from easyeditor import KnowEditDataset

In [2]:
data_dir = '../dataset/ccks2024_know_edit/ZsRE-test-all.json'
train_data_path = None
ds_size, data_type, = 10, 'zsre'
hparams_dir = '../hparams/LoRA/gpt-neo-125m.yaml'
metrics_save_dir = './EasyEditCache/metrics'

In [3]:
datas = KnowEditDataset(data_dir,size=ds_size)
if data_type == 'counterfact' or data_type == 'recent' or data_type == 'zsre':
    prompts=[data['prompt'] for data in datas]
    subjects=[data['subject'] for data in datas]
    target_new = [data['target_new'] for data in datas]
    
    portability_r =[data['portability_r'] for data in datas]
    portability_s =[data['portability_s'] for data in datas]
    portability_l =[data['portability_l'] for data in datas]

    portability_reasoning_prompts=[]
    portability_reasoning_ans=[]
    portability_Logical_Generalization_prompts=[]
    portability_Logical_Generalization_ans=[]
    portability_Subject_Aliasing_prompts=[]
    portability_Subject_Aliasing_ans=[]
    
    portability_data = [portability_r,portability_s,portability_l]
    portability_prompts = [portability_reasoning_prompts,portability_Subject_Aliasing_prompts,portability_Logical_Generalization_prompts]
    portability_answers = [portability_reasoning_ans,portability_Subject_Aliasing_ans,portability_Logical_Generalization_ans]
    for data, portable_prompts, portable_answers in zip(portability_data,portability_prompts,portability_answers):
        for item in data:
            if item is None:
                portable_prompts.append(None)
                portable_answers.append(None)
            else:
                temp_prompts = []
                temp_answers = []
                for pr in item:
                    prompt=pr["prompt"]
                    an=pr["ground_truth"]
                    while isinstance(an,list):
                        an = an[0]
                    if an.strip() =="":
                        continue
                    temp_prompts.append(prompt)
                    temp_answers.append(an)
                portable_prompts.append(temp_prompts)
                portable_answers.append(temp_answers)
    assert len(prompts) == len(portability_reasoning_prompts) == len(portability_Logical_Generalization_prompts) == len(portability_Subject_Aliasing_prompts)
    
    locality_rs = [data['locality_rs'] for data in datas]
    locality_f = [data['locality_f'] for data in datas]
    locality_Relation_Specificity_prompts=[]
    locality_Relation_Specificity_ans=[]
    locality_Forgetfulness_prompts=[]        
    locality_Forgetfulness_ans=[]
    
    locality_data = [locality_rs, locality_f]
    locality_prompts = [locality_Relation_Specificity_prompts,locality_Forgetfulness_prompts]
    locality_answers = [locality_Relation_Specificity_ans,locality_Forgetfulness_ans]
    for data, local_prompts, local_answers in zip(locality_data,locality_prompts,locality_answers):
        for item in data:
            if item is None:
                local_prompts.append(None)
                local_answers.append(None)
            else:
                temp_prompts = []
                temp_answers = []
                for pr in item:
                    prompt=pr["prompt"]
                    an=pr["ground_truth"]
                    while isinstance(an,list):
                        an = an[0]
                    if an.strip() =="":
                        continue
                    temp_prompts.append(prompt)
                    temp_answers.append(an)
                local_prompts.append(temp_prompts)
                local_answers.append(temp_answers)
    assert len(prompts) == len(locality_Relation_Specificity_prompts) == len(locality_Forgetfulness_prompts)
    locality_inputs = {}
    portability_inputs = {}
    
    locality_inputs = {
        'Relation_Specificity':{
            'prompt': locality_Relation_Specificity_prompts,
            'ground_truth': locality_Relation_Specificity_ans
        },
        'Forgetfulness':{
            'prompt':locality_Forgetfulness_prompts,
            'ground_truth':locality_Forgetfulness_ans
        }
    }
    portability_inputs = {
        'Subject_Aliasing':{
            'prompt': portability_Subject_Aliasing_prompts,
            'ground_truth': portability_Subject_Aliasing_ans
        },
        'reasoning':{
            'prompt': portability_reasoning_prompts,
            'ground_truth': portability_reasoning_ans           
        },
        'Logical_Generalization':{
            'prompt': portability_Logical_Generalization_prompts,
            'ground_truth': portability_Logical_Generalization_ans           
        }
    }
if data_type == 'wikibio':
    prompts=[data['prompt'] for data in datas]
    subjects=[data['subject'] for data in datas]
    target_new = [data['target_new'] for data in datas]
    
    locality_rs = [data['locality_rs'] for data in datas]
    locality_f = [data['locality_f'] for data in datas]
    locality_Relation_Specificity_prompts=[]
    locality_Relation_Specificity_ans=[]
    
    locality_data = [locality_rs]
    locality_prompts = [locality_Relation_Specificity_prompts]
    locality_answers = [locality_Relation_Specificity_ans]
    for data, local_prompts, local_answers in zip(locality_data,locality_prompts,locality_answers):
        for item in data:
            if item is None:
                local_prompts.append(None)
                local_answers.append(None)
            else:
                temp_prompts = []
                temp_answers = []
                for pr in item:
                    prompt=pr["prompt"]
                    an=pr["ground_truth"]
                    while isinstance(an,list):
                        an = an[0]
                    if an.strip() =="":
                        continue
                    temp_prompts.append(prompt)
                    temp_answers.append(an)
                local_prompts.append(temp_prompts)
                local_answers.append(temp_answers)
    assert len(prompts) == len(locality_Relation_Specificity_prompts)
    portability_inputs = None
    locality_inputs = {}
    locality_inputs = {
        'Relation_Specificity':{
            'prompt': locality_Relation_Specificity_prompts,
            'ground_truth': locality_Relation_Specificity_ans
        }
    }

hparams = LoRAHyperParams.from_hparams(hparams_dir)
pre_file = f"../pre_edit/{hparams.model_name.split('/')[-1]}_{data_type}_pre_edit.json"
if pre_file is not None and os.path.exists(pre_file):
    pre_edit = json.load(open(pre_file,'r'))
    assert len(pre_edit) == len(prompts)
else:
    pre_edit = None

train_ds = None

In [4]:
editor = BaseEditor.from_hparams(hparams)

2024-07-14 21:18:33,365 - easyeditor.editors.editor - INFO - Instantiating model
07/14/2024 21:18:33 - INFO - easyeditor.editors.editor -   Instantiating model
2024-07-14 21:18:33,985 - easyeditor.editors.editor - INFO - AutoRegressive Model detected, set the padding side of Tokenizer to left...
07/14/2024 21:18:33 - INFO - easyeditor.editors.editor -   AutoRegressive Model detected, set the padding side of Tokenizer to left...


In [5]:
# 单条数据编辑
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    target_new=target_new,
    subject=subjects,
    locality_inputs=locality_inputs,
    portability_inputs=portability_inputs,
    train_ds=train_ds,
    keep_original_weight=True,
    pre_file=pre_file,
    pre_edit = pre_edit,
    test_generation=True,
)

  0%|          | 0/10 [00:00<?, ?it/s]

trainable params: 245,760 || all params: 125,444,352 || trainable%: 0.19591157041490398
Executing LoRA algo for: [Which family does Epaspidoceras belong to?] -> [Noctuidae]
Using device: cuda:0
Epoch: 0
Batch loss 5.988340854644775
Total loss 5.988340854644775
Epoch: 1
Batch loss 5.5630669593811035
Total loss 5.5630669593811035
Epoch: 2
Batch loss 3.172980546951294
Total loss 3.172980546951294


2024-07-14 21:31:00,578 - easyeditor.editors.editor - INFO - 0 editing: Which family does Epaspidoceras belong to? -> Noctuidae  

 {'pre': {'rewrite_acc': [0.3333333333333333], 'portability': {'reasoning_acc': [0.0]}, 'fluency': {'ngram_entropy': 6.019608905037443}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Which family does Epaspidoceras belong to?', 'target_new': 'Noctuidae', 'ground_truth': '<|endoftext|>', 'portability': {'reasoning': {'prompt': ['What is the common name for the family Epaspidoceras belongs to?'], 'ground_truth': ['Owlet moths']}}, 'locality': {'Relation_Specificity': {'prompt': ['The taxon rank of Epaspidoceras is', 'Epaspidoceras taxon rank'], 'ground_truth': ['genus', 'genus']}}, 'subject': 'Epaspidoceras'}, 'post': {'rewrite_acc': [1.0], 'locality': {'Relation_Specificity_acc': [1.0, 1.0]}, 'portability': {'reasoning_acc': [0.0]}, 'fluency': {'ngram_entropy': 6.1231840489833855}}}
07/14/2024 21:31:00 - INFO - easyeditor.editors.editor -   0 editing: Whic

trainable params: 245,760 || all params: 125,444,352 || trainable%: 0.19591157041490398
Executing LoRA algo for: [What species is ZIC3 specific to?] -> [male]
Using device: cuda:0
Epoch: 0



No chat template is defined for this tokenizer - using the default template for the GPT2Tokenizer class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



...

trainable params: 245,760 || all params: 125,444,352 || trainable%: 0.19591157041490398
Executing LoRA algo for: [What day was USA-199 launched?] -> [20 December 2007]
Using device: cuda:0
Epoch: 0
Batch loss 6.395610332489014
Total loss 6.395610332489014
Epoch: 1
Batch loss 6.1013875007629395
Total loss 6.1013875007629395
Epoch: 2
Batch loss 3.734497308731079
Total loss 3.734497308731079


2024-07-14 21:49:15,301 - easyeditor.editors.editor - INFO - 8 editing: What day was USA-199 launched? -> 20 December 2007  

 {'pre': {'rewrite_acc': [0.0], 'portability': {'Subject_Aliasing_acc': [0.0]}, 'fluency': {'ngram_entropy': 6.2902881894375975}}, 'case_id': 8, 'requested_rewrite': {'prompt': 'What day was USA-199 launched?', 'target_new': '20 December 2007', 'ground_truth': '<|endoftext|>', 'portability': {'Subject_Aliasing': {'prompt': ['When was GPS IIR-18(M) launched?'], 'ground_truth': ['20 December 2007']}}, 'locality': {'Relation_Specificity': {'prompt': ['The country of USA-199 is', 'USA-199 country'], 'ground_truth': ['United States of America', 'United States of America']}}, 'subject': 'USA-199'}, 'post': {'rewrite_acc': [1.0], 'locality': {'Relation_Specificity_acc': [0.75, 0.75]}, 'portability': {'Subject_Aliasing_acc': [0.3333333333333333]}, 'fluency': {'ngram_entropy': 5.878844342749822}}}
07/14/2024 21:49:15 - INFO - easyeditor.editors.editor -   8 editing: What

trainable params: 245,760 || all params: 125,444,352 || trainable%: 0.19591157041490398
Executing LoRA algo for: [What was the record label of Runaway Sunday?] -> [Motown]
Using device: cuda:0
Epoch: 0
Batch loss 6.480423927307129
Total loss 6.480423927307129
Epoch: 1
Batch loss 6.032350540161133
Total loss 6.032350540161133
Epoch: 2
Batch loss 3.569950580596924
Total loss 3.569950580596924


2024-07-14 21:49:16,867 - easyeditor.editors.editor - INFO - 9 editing: What was the record label of Runaway Sunday? -> Motown  

 {'pre': {'rewrite_acc': [0.5], 'portability': {'reasoning_acc': [0.6666666666666666]}, 'fluency': {'ngram_entropy': 6.165015710449331}}, 'case_id': 9, 'requested_rewrite': {'prompt': 'What was the record label of Runaway Sunday?', 'target_new': 'Motown', 'ground_truth': '<|endoftext|>', 'portability': {'reasoning': {'prompt': ['Who founded the record label that signed Runaway Sunday?'], 'ground_truth': ['Berry Gordy']}}, 'locality': {'Relation_Specificity': {'prompt': ['The distribution format of Runaway Sunday is', 'Runaway Sunday distribution format'], 'ground_truth': ['music streaming', 'music streaming']}}, 'subject': 'Runaway Sunday'}, 'post': {'rewrite_acc': [1.0], 'locality': {'Relation_Specificity_acc': [0.5, 1.0]}, 'portability': {'reasoning_acc': [0.6666666666666666]}, 'fluency': {'ngram_entropy': 2.0132979349379063}}}
07/14/2024 21:49:16 - INFO -

Metrics Summary:  {'pre': {'rewrite_acc': 0.25833333333333336, 'portability': {'reasoning_acc': 0.2733333333333333, 'Subject_Aliasing_acc': 0.0, 'Logical_Generalization_acc': 0.4166666666666667}}, 'post': {'rewrite_acc': 0.905, 'locality': {'Relation_Specificity_acc': 0.85}, 'portability': {'reasoning_acc': 0.3, 'Subject_Aliasing_acc': 0.6666666666666666, 'Logical_Generalization_acc': 0.49074074074074076}}}


In [ ]:
# 批量数据编辑
metrics, edited_model, _ = editor.batch_edit(
    prompts=prompts,
    target_new=target_new,
    subject=subjects,
    locality_inputs=locality_inputs,
    portability_inputs=portability_inputs,
    train_ds=train_ds,
    keep_original_weight=True,
    pre_file=pre_file,
    pre_edit = pre_edit,
    test_generation=True,
)

In [ ]:
if not os.path.exists(metrics_save_dir):
    os.makedirs(metrics_save_dir)
json.dump(metrics, open(os.path.join(metrics_save_dir, f'{editing_method}_{data_type}_{hparams_dir.split("/")[-1]}_results.json'), 'w'), indent=4)
